In [5]:
import os

In [6]:
%pwd

'c:\\Users\\fifty\\OneDrive\\Desktop\\AI - Data Science\\Mlops\\Malicious_QR_Code_Detection\\research'

In [7]:
os.chdir('../')

In [8]:
%pwd

'c:\\Users\\fifty\\OneDrive\\Desktop\\AI - Data Science\\Mlops\\Malicious_QR_Code_Detection'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    model_path: str
    train_data: str
    test_data: str
    train_metrics_file: str
    test_metrics_file: str

In [10]:
from src.MaliciousQRCodeDetection.constants import *
from src.MaliciousQRCodeDetection.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 ):
        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            model_path = config.model_path,
            train_data = config.train_data,
            test_data=config.test_data,
            train_metrics_file=config.train_metrics_file,
            test_metrics_file=config.test_metrics_file,
          
        )

        return model_evaluation_config

In [12]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score, roc_curve
import json
import pandas as pd
from tensorflow.keras.models import load_model

In [13]:
from src.MaliciousQRCodeDetection.logging.logger import logger
from src.MaliciousQRCodeDetection.exception import MaliciousQRException

In [24]:
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config = config

    def metrics(self):

        train_df = pd.read_csv(self.config.train_data)
        test_df = pd.read_csv(self.config.test_data)
        model =  load_model(self.config.model_path)
        logger.info('Train data, test data and model loaded sucessfully ..')

        X_train = train_df.drop(columns=['result'])
        y_train = train_df['result']
        X_test = test_df.drop(columns=['result'])
        y_test = test_df['result']

        # Handle missing values
        logger.info(f"Number of NaNs in y_train: {y_train.isna().sum()}")
        logger.info(f"Number of NaNs in y_test: {y_test.isna().sum()}")
        logger.info(f"y_test shape: {y_test.shape}")

        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        y_train_binary = [0 if prob < 0.5 else 1 for prob in y_train_pred]
        y_test_binary = [0 if prob < 0.5 else 1 for prob in y_test_pred]

        train_json = {
            'Accuracy Score: ': accuracy_score(y_train,y_train_binary),        
        }

        test_json = {
            'Accuracy Score: ': accuracy_score(y_test,y_test_binary),
        }

        with open(self.config.train_metrics_file, 'w') as f:
            json.dump(train_json, f)

        with open(self.config.test_metrics_file, 'w') as f:
            json.dump(test_json, f)

        
        logger.info('Train & Test Metrics Json file saved sucessfully.')





In [1]:
import sys

In [25]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.metrics()
    
except Exception as e:
    logger.info('Error occured in model evaluation section..')
    raise MaliciousQRException(e,sys)

[ 2024-12-29 15:47:41,356] 17 root - INFO - yaml file: config\config.yaml loaded successfully
[ 2024-12-29 15:47:41,357] 31 root - INFO - Created directory at: artifacts
[ 2024-12-29 15:47:41,358] 31 root - INFO - Created directory at: artifacts/model_evaluation
[ 2024-12-29 15:47:43,455] 10 root - INFO - Train data, test data and model loaded sucessfully ..
[ 2024-12-29 15:47:43,495] 18 root - INFO - Number of NaNs in y_train: 0
[ 2024-12-29 15:47:43,496] 19 root - INFO - Number of NaNs in y_test: 0
[ 2024-12-29 15:47:43,497] 20 root - INFO - y_test shape: (126502,)
15813/15813 ━━━━━━━━━━━━━━━━━━━━ 9s 566us/step
3954/3954 ━━━━━━━━━━━━━━━━━━━━ 2s 550us/step
[ 2024-12-29 15:47:58,488] 43 root - INFO - Train & Test Metrics Json file saved sucessfully.
